# 实现RNN

In [1]:
#encoding:utf-8
import json
import time
import random
import numpy as np
import pandas as pd
import math
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import sys
from sklearn.externals import joblib
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import time
from torch.autograd import Variable
from glob import glob
import os
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import shutil
from torchvision import transforms
from torchvision import models
from torchtext import data, datasets
from nltk import ngrams
from torchtext.vocab import GloVe, Vectors
from collections import defaultdict
data_path = r'D:\kaggle\data\spooky-author-identification\a'[: -1]
data_path_inv = r'D:\kaggle\data\spooky-author-identification\a'[: -1]
data_path_word_vector = r'D:\kaggle\data\word_vector\a'[: -1]

import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
stop_words = stopwords.words('english')

In [2]:
def read_data():
    df_train = pd.read_csv(data_path_inv + r'train/train.csv')
    df_test = pd.read_csv(data_path + 'test/test.csv')
    df_sub = pd.read_csv(data_path + 'sample_submission/sample_submission.csv')
    return df_train, df_test, df_sub
df_train, df_test, df_sub = read_data()
print (df_train.shape, df_test.shape, df_sub.shape)
print (df_train.head(3), '\n\n', df_test.head(3), '\n\n', df_sub.head(3), )

(19579, 3) (8392, 2) (8392, 4)
        id                                               text author
0  id26305  This process, however, afforded me no means of...    EAP
1  id17569  It never once occurred to me that the fumbling...    HPL
2  id11008  In his left hand was a gold snuff box, from wh...    EAP 

         id                                               text
0  id02310  Still, as I urged our leaving Ireland with suc...
1  id24541  If a fire wanted fanning, it could readily be ...
2  id00134  And when they had broken down the frail door t... 

         id       EAP       HPL       MWS
0  id02310  0.403494  0.287808  0.308698
1  id24541  0.403494  0.287808  0.308698
2  id00134  0.403494  0.287808  0.308698


In [3]:
# 处理数据
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(df_train.author.values)
xtrain, xvalid, ytrain, yvalid = train_test_split(df_train.text.values, y, 
                                                  stratify = y, 
                                                  random_state = 2020, 
                                                  test_size = 0.1, shuffle = True)
xtest = df_test.text.values
print (xtrain.shape)
print (xvalid.shape)
print (xtest.shape)

(17621,)
(1958,)
(8392,)


In [4]:
# load the GloVe vectors in a dictionary:
embeddings_index = {}
f = open(data_path_word_vector + 'glove.840B.300d.txt', 'rb')
index = 0
pre_time = time.time()
for line in f: # tqdm(f):
    index += 1
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    if index % 500000 == 0:
        print ('index: {:}, time:{:}'.format(index, time.time() - pre_time))
        pre_time = time.time()
f.close()
print('Found %s word vectors.' % len(embeddings_index))

index: 100000, time:8.498717784881592
index: 200000, time:8.46131181716919
index: 300000, time:8.484855890274048
index: 400000, time:8.412349224090576
index: 500000, time:8.444247245788574
index: 600000, time:8.373737335205078
index: 700000, time:8.501246452331543
index: 800000, time:8.392608880996704
index: 900000, time:8.472589254379272
index: 1000000, time:8.454006671905518
index: 1100000, time:8.509315252304077
index: 1200000, time:8.338173866271973
index: 1300000, time:8.497522830963135
index: 1400000, time:8.592368841171265
index: 1500000, time:8.432148456573486
index: 1600000, time:8.278294324874878
index: 1700000, time:8.268243312835693
index: 1800000, time:8.38856053352356
index: 1900000, time:8.368398427963257
index: 2000000, time:8.25022554397583
index: 2100000, time:8.356012105941772
Found 2196016 word vectors.


In [5]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            torch_tmp = torch.from_numpy(np.array(embeddings_index[str.encode(w)])).cuda().type(torch.float32)
            M.append(torch_tmp)
        except:
            continue
    M = np.array(M)
    # v = M.sum(axis=0)
    v = M
    if type(v) != np.ndarray:
        return np.zeros((1, 300))
    return v

xtrain_glove = np.array([sent2vec(x) for x in xtrain])
print (xtrain_glove.shape, type(xtrain_glove[0]), xtrain_glove[0].shape)
xvalid_glove = np.array([sent2vec(x) for x in xvalid])
print (xvalid_glove.shape, type(xvalid_glove[0]), xtrain_glove[0].shape)
xtest_glove = np.array([sent2vec(x) for x in xtest])
print (xtest_glove.shape, type(xtest_glove[0]), xtrain_glove[0].shape)

# # scale the data before any neural net:
# scl = preprocessing.StandardScaler()
# xtrain_glove_scl = scl.fit_transform(xtrain_glove)
# xvalid_glove_scl = scl.transform(xvalid_glove)

print (xtrain_glove.shape, xvalid_glove.shape)
print (ytrain.shape, yvalid.shape)
print (ytrain[: 3], yvalid[: 3])
word_vector_size = len(embeddings_index[list(embeddings_index.keys())[0]])
label_size = 3

(17621,) <class 'numpy.ndarray'> (17,)
(1958,) <class 'numpy.ndarray'> (17,)
(8392,) <class 'numpy.ndarray'> (17,)
(17621,) (1958,)
(17621,) (1958,)
[1 0 1] [1 1 1]


In [6]:
num_inputs, num_hiddens, num_outputs = word_vector_size, 512, 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)

    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])

def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
    Y = torch.matmul(H, W_hq) + b_q
    # Y = Y.view(-1)
    return Y

def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

def data_iter_fn(xtrain, ytrain, batch_size, device):
    num_steps = len(xtrain_glove_scl) // batch_size
    x_out, y_out = [], []
    for epoch in range(num_steps):
        xval = torch.from_numpy(xtrain[i * batch_size: (i + 1) * batch_size, :]).cuda().type(torch.float32)
        yval = ytrain[i * batch_size: (i + 1) * batch_size]
        x_out.append(xval)
        y_out.append(yval)
    y_out = torch.from_numpy(np.array(y_out)).cuda().type(torch.float32)
    return x_out, y_out

def sgd(params, lr, batch_size):
    # 为了和原书保持一致，这里除以了batch_size，但是应该是不用除的，因为一般用PyTorch计算loss时就默认已经
    # 沿batch维求了平均了。
    for param in params:
        param.data -= lr * param.grad / batch_size # 注意这里更改param时用的param.data

will use cuda


In [ ]:
# 定义模型训练函数
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train(xtrain, ytrain, rnn, get_params, init_rnn_state, num_hiddens,
                      word_vector_size, device, num_epochs, lr, clipping_theta, batch_size):
    params = get_params()
    loss = nn.CrossEntropyLoss()
    l_sum, n, start = 0.0, 0, time.time()
    for epoch in range(num_epochs):
        outputs = []
        for X in xtrain:
            state = init_rnn_state(batch_size, num_hiddens, device)
            Y = rnn(X, state, params)
            # val = Y.cpu().detach().numpy().reshape(-1)
            # print (Y.shape)
            outputs.append(Y)
        outputs = torch.cat(outputs, dim=0)
        # Y的形状是(batch_size, num_steps)，转置后再变成长度为
        # batch * num_steps 的向量，这样跟输出的行一一对应
        y = torch.from_numpy(ytrain).cuda().type(torch.float32)
        l = loss(outputs, y.long())

        # 梯度清0
        if params[0].grad is not None:
            for param in params:
                param.grad.data.zero_()
        l.backward()
        grad_clipping(params, clipping_theta, device)  # 裁剪梯度
        sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
        l_sum += l.item() * y.shape[0]
        n += y.shape[0]
        
        pred_period = 1
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
    return params

# 训练模型
num_epochs, lr, clipping_theta, batch_size = 15, 1e2, 1e-2, 1
params = train(xtrain_glove, ytrain, rnn, get_params, init_rnn_state, num_hiddens,
                      word_vector_size, device, num_epochs, lr, clipping_theta, batch_size)

epoch 1, perplexity 3.001179, time 201.09 sec


In [ ]:
def get_outputs(inputs, params):
    outputs = []
    for X in inputs:
        state = init_rnn_state(batch_size, num_hiddens, device)
        Y = rnn(X, state, params)
        outputs.append(Y)
    outputs = torch.cat(outputs, dim=0)
    return outputs

def validation(xvalidation, yvalidation, params):
    outputs = get_outputs(xvalidation, params)
    y_pred = torch.max(outputs, 1)[1].cpu().numpy()
    y_val = yvalidation
    print (y_pred.shape, y_val.shape)
    cnt_all = len(y_pred)
    cnt_correct = (y_pred == y_val).sum()
    print ("acc : {:} / {:} = {:.3f}".format(cnt_correct, cnt_all, cnt_correct * 1.0 / cnt_all))

validation(xvalid_glove, yvalid, params)

# online

In [ ]:
outputs = get_outputs(xtest_glove, params)
y_out = F.softmax(outputs, dim = 1).cpu().detach().numpy()
df_sub_out = pd.DataFrame(y_out, columns = ['EAP', 'HPL', 'MWS'])
df_sub_out['id'] = df_test['id']
df_sub_out[['id', 'EAP', 'HPL', 'MWS']].to_csv(data_path_inv + 'submission.csv', index = False, header = True)
print (len(df_sub_out), len(df_sub))

In [ ]:
print (df_sub_out.head(3), df_sub.head(3))